In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk
import re

# Spacy
import spacy
nlp=spacy.load('es_core_news_sm')

# Stemmer
from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')

# Levantamos la lista de StopWords
f = open('stopwords_intents.txt', 'r', encoding='utf8')
stopwords = f.read().split('\n')
f.close()
len(stopwords)

265

In [2]:
def PreProcesar(Corpus, POS=False, Lema=True, Stem=True):
    
    
    # Generar una lista de documentos de spacy para tratar el POS Tagging y la Lematización
    docs=[]
    for oracion in Corpus:
        docs.append(nlp(oracion.lower())) #La lematización funciona mejor en minúsculas
    
    # Crear una lista de oraciones, donde cada elemento es una lista de palabras.
    # Cada palabra está definida por una tupla (Texto, POSTag, Lema)
    # Se omiten los tokens que son identificados como signos de puntuación
    oraciones=[]
    for doc in docs:
        oracion=[]
        for token in doc:
            if token.pos_ != 'PUNCT':
                oracion.append((token.text, token.pos_, token.lemma_))
        oraciones.append(oracion)
    
    # Removemos StopWords (finándonos en el lema de cada palabra en vez de su texto!)
    # No conviene quitar las StopWords antes de lematizar pues son útiles para ese proceso...
    oraciones = [[palabra for palabra in oracion if palabra[2] not in stopwords] for oracion in oraciones]
    
    # Stemming
    if Stem==True:
        oraciones_aux=[]
        for oracion in oraciones:
            oracion_aux=[]
            for palabra in oracion:
                p_texto, p_pos, p_lema = palabra
                # Si Lema es True, se Stemmatiza el lema; si no, se Stemmatiza la palabra original
                if Lema==True:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_lema)))
                else:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_texto)))
            oraciones_aux.append(oracion_aux)
        
        oraciones = oraciones_aux
    
    # Finalmente: devolver nuevamente una lista de cadenas como la recibida, pero con el contenido
    # de cada cadena conformado según los parámetros:
    
    Corpus_Procesado = [] #Variable de salida
    
    for doc in oraciones:
        oracion = ''
        for palabra in doc:
            if Stem == True:
                # Devolver cadena de Stemming
                oracion = oracion + palabra[3]
            else:
                if Lema == True:
                    # Devolver cadena de Lemas
                    oracion = oracion + palabra[2]
                else:
                    # Devolver cadena de palabras originales
                    oracion = oracion + palabra[0]
            
            if POS == True:
                #Concatenar POS a cada palabra
                oracion = oracion + '_' + palabra[1].lower()
            
            oracion = oracion + ' '
        
        Corpus_Procesado.append(oracion)
        
    return Corpus_Procesado

def Corregir_Documentos(df_textos, columnas, POS=False, Lema=True, Stem=True):

    for col in columnas:
        df_textos[col] = PreProcesar(list(df_textos[col]), POS, Lema, Stem)
    
    df_textos = df_textos.drop_duplicates().reset_index(drop=True)
    
    return df_textos

def Generar_Matriz_BOW(df_textos, columna, binario=False, ngram=(1,2)):
    
    # Vectorizar, usando CountVectorizer de sklearn.feature_extraction.text
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizador = CountVectorizer(binary=binario, ngram_range=ngram)
    X = vectorizador.fit_transform(df_textos[columna])
    
    # Generar el DataFrame a devolver
    df_X = pd.DataFrame(X.toarray(), columns=vectorizador.get_feature_names())
    df = df_textos.join(df_X)
    
    return vectorizador, df

def Generar_Matriz_Tfidf(df_textos, columna, ngram=(1,2)):
    
    # Vectorizar... Directamente usar aquí el TfidfVectorizer de sklearn en vez del CountVectorizer
    # (Lleva los mismos parámetros y directamente nos devuelve la matriz con los vectores Tf*Idf)
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizador = TfidfVectorizer(ngram_range=ngram)
    X = vectorizador.fit_transform(df_textos[columna])
    
    # Generar el DataFrame a devolver
    df_X = pd.DataFrame(X.toarray(), columns=vectorizador.get_feature_names())
    df = df_textos.join(df_X)
    
    return vectorizador, df

def Distancia_Coseno(u, v):
    
    distancia = 1.0 - (np.dot(u, v) / (np.sqrt(sum(np.square(u))) * np.sqrt(sum(np.square(v)))))
    return distancia

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

#1. Cargar y corregir el corpus
df_textos = pd.read_csv('data_intents.csv', sep=';', encoding='utf_8')
df_textos = Corregir_Documentos(df_textos,['oracion'],False,True,True)

#2. Modelizar los documentos de df_textos
vectorizador, df_textos = Generar_Matriz_Tfidf(df_textos,'oracion',ngram=(1,3))
#vectorizador, df_textos = Generar_Matriz_BOW(df_textos,'oracion')

#3. Separar el corpus en Train/Test
X = df_textos.drop(["intencion","subIntencion"],axis=1)
y = df_textos[["intencion"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=124)
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


In [4]:
#si paso dos listas a zip como entrada, el resultado será una tupla 
#donde cada elemento tendrá cada uno de los elementos pasados como entrada

#df_textos['intenciones'] = df_textos[['intencion', 'subIntencion']].apply(tuple, axis=1)
#df_textos["ff"]=list(df_textos[['intencion', 'subIntencion']].itertuples(index=False, name=None))
df_textos_tupla = df_textos.copy()
df_textos_tupla["intenciones"] = list(zip(df_textos_tupla.intencion, df_textos_tupla.subIntencion))
df_textos_tupla.drop(["intencion","subIntencion"],axis=1, inplace=True)
df_textos_tupla

,oracion,intenciones
0,qué es el ies?,"(charla, todas)"
1,cuál es la dirección de ies?,"(charla, todas)"
2,cuál es la dirección de ies?,"(charla, todas)"
3,cuál es su dirección?,"(charla, todas)"
4,dónde queda el ies?,"(charla, todas)"
...,...,...
935,quiero estudiar algo con rapida salida laboral,"(trabajo, todas)"
936,quiero estudiar algo para ganar plata,"(trabajo, todas)"
937,que requisitos tiene la cerrera,"(generalidades, todas)"
938,me faltan materias del secundario ¿me puedo an...,"(tramites, requisitos)"


In [4]:
df_comparativa = pd.DataFrame(columns=['Modelo','Umbral','Aciertos',
'Errores','Indeterm','Precision','Recall','Accuracy','F1','df_Aciertos','df_Errores','df_Indeterm'])

#Distancia_Coseno del profe

def Evaluar_Modelo_Distancia_Coseno(X_train, X_test, y_test, umbral=0.8):
    # Recorrer todo el Test Set prediciendo para cada oración de X_test y comparando con y_test
    global df_comparativa
    df_test = X_test[['oracion']].join(y_test)
    array_X = X_train[X_train.columns[1:]].values
    lista_distancia = [] #Distancia con el elemento más cercano (predicción)
    lista_predic    = [] #Predicción del elemento más cercano
    lista_similar   = [] #Texto del elemento más cercano
    for test_doc in df_test.iterrows():
        i,d = test_doc
        df_query = pd.DataFrame([d['oracion']],columns=['oracion'])
        Q = vectorizador.transform(df_query['oracion'])
        distancia = [Distancia_Coseno(Q.A[0],fila) for fila in array_X]
        df_Resultado = pd.DataFrame(distancia, columns=['Distancia']).join(X_train[['oracion']]).join(y_train[['intencion']]).sort_values(by='Distancia').head(1)

        lista_distancia.append(df_Resultado.iloc[0]['Distancia'])
        lista_predic.append(df_Resultado.iloc[0]['intencion']) 
        lista_similar.append(df_Resultado.iloc[0]['oracion'])

    # Agregar columnas con resultados predichos al df_test
    df_test['Distancia'] = lista_distancia
    df_test['Predic']    = lista_predic
    df_test['Similar']   = lista_similar

    # Evaluar el resultado en df_test
    print('Cantidad de registros evaluados:', len(df_test))
    print('--------------------')
    aciertos = df_test[ ( df_test['intencion'] == df_test['Predic'] ) & 
                        ( df_test['Distancia'] <= umbral ) ]['intencion'].count()
    errores  = df_test[ ( df_test['intencion'] != df_test['Predic'] ) & 
                        ( df_test['Distancia'] <= umbral) ]['intencion'].count()
    indeterm = df_test[ ( df_test['Distancia'] > umbral) ]['Distancia'].count()
    print('Aciertos:', aciertos)
    print('Errores :', errores)
    print('Indeterm:', indeterm)
    print('--------------------')
    precision = aciertos/(aciertos+errores)
    recall    = aciertos/(aciertos+indeterm)
    accuracy  = (aciertos+indeterm)/(aciertos+errores+indeterm)
    F1        = 2*((precision*recall)/(precision+recall))
    print('Precision: {0:.3f} <- aciertos/(aciertos+errores)'.format(precision))
    print('Recall   : {0:.3f} <- aciertos/(aciertos+indeterm)'.format(recall))
    print('Accuracy : {0:.3f} <- (aciertos+indeterm)/(aciertos+errores+indeterm)'.format(accuracy))
    print('F1       : {0:.3f} <- 2*((precision*recall)/(precision+recall))'.format(F1))

    df_comparativa = df_comparativa.append({'Modelo': 'Distancia coseno',
                                            'Umbral': umbral,
                                            'Aciertos': aciertos,
                                            'Errores': errores,
                                            'Indeterm': indeterm,
                                            'Precision': precision,
                                            'Recall': recall,
                                            'Accuracy': accuracy,
                                            'F1': F1,
                                            'df_Aciertos': df_test[ ( df_test['intencion'] == df_test['Predic'] ) & ( df_test['Distancia'] <= umbral ) ],
                                            'df_Errores' : df_test[ ( df_test['intencion'] != df_test['Predic'] ) & ( df_test['Distancia'] <= umbral ) ],
                                            'df_Indeterm': df_test[ ( df_test['Distancia'] > umbral ) ]
                                            }, ignore_index=True)
    
    return

In [5]:
Evaluar_Modelo_Distancia_Coseno(X_train, X_test, y_test, umbral=0.8)

Cantidad de registros evaluados: 157
--------------------
Aciertos: 100
Errores : 36
Indeterm: 21
--------------------
Precision: 0.735 <- aciertos/(aciertos+errores)
Recall   : 0.826 <- aciertos/(aciertos+indeterm)
Accuracy : 0.771 <- (aciertos+indeterm)/(aciertos+errores+indeterm)
F1       : 0.778 <- 2*((precision*recall)/(precision+recall))


In [6]:
from sklearn.linear_model import LogisticRegression

# Crear el modelo con los parámetros que no cambiarán: observar que no le pasamos el valor de C de regulrización
RLog=LogisticRegression(penalty='none', max_iter=10000, tol=0.0001, multi_class='ovr',)

# Armar el diccionario con el nombre y valores para los Hiperparámetros
parametros_RLog = {'C':[1]}

# Armar el GridSearchCV
grid_RLog = GridSearchCV(estimator = RLog,scoring = 'accuracy',param_grid = parametros_RLog, cv = 5,
                        n_jobs = -1)

# Entrenar con el Train Set
grid_RLog.fit(X_train[X_train.columns[1:]].values, y_train);

# Obtener el mejor AC 
AC_RLog_best=grid_RLog.best_score_
print('Mejor accuracy: ' + str(round(AC_RLog_best,4)))

C_RLog_best=grid_RLog.best_params_ 
print('Mejor C: ' + str(C_RLog_best))
###########################################################################################################
def Evaluar_Modelo(modelo, nombre_modelo, X_test, y_test, umbral=0.7):
    # Recorrer todo el Test Set prediciendo para cada oración de X_test y comparando con y_test
    global df_comparativa
    df_test = X_test[['oracion']].join(y_test)
    lista_predic       = [] #Predicción
    lista_probabilidad = [] #Probabilidad de la predicción
    for test_doc in df_test.iterrows():
        i,d = test_doc
        df_query = pd.DataFrame([d['oracion']],columns=['oracion'])
        Q = vectorizador.transform(df_query['oracion'])
        pronostico = modelo.predict([Q.A[0]])
        probabilidad = modelo.predict_proba([Q.A[0]])
        lista_predic.append(pronostico[0])
        lista_probabilidad.append(probabilidad[0].max())
    
    # Agregar columnas con resultados predichos al df_test
    df_test['Probabilidad'] = lista_probabilidad
    df_test['Predic'] = lista_predic
    
    # Evaluar el resultado en df_test
    print('Cantidad de registros evaluados:', len(df_test))
    print('--------------------')
    aciertos = df_test[ ( df_test['intencion'] == df_test['Predic'] ) & 
                        ( df_test['Probabilidad'] >= umbral ) ]['intencion'].count()
    errores  = df_test[ ( df_test['intencion'] != df_test['Predic'] ) & 
                        ( df_test['Probabilidad'] >= umbral) ]['intencion'].count()
    indeterm = df_test[ ( df_test['Probabilidad'] < umbral) ]['Probabilidad'].count()
    
    print('Aciertos:', aciertos)
    print('Errores :', errores)
    print('Indeterm:', indeterm)
    print('--------------------')
    precision = aciertos/(aciertos+errores)
    recall    = aciertos/(aciertos+indeterm)
    accuracy  = (aciertos+indeterm)/(aciertos+errores+indeterm)
    F1        = 2*((precision*recall)/(precision+recall))
    print('Precision: {0:.3f} <- aciertos/(aciertos+errores)'.format(precision))
    print('Recall   : {0:.3f} <- aciertos/(aciertos+indeterm)'.format(recall))
    print('Accuracy : {0:.3f} <- (aciertos+indeterm)/(aciertos+errores+indeterm)'.format(accuracy))
    print('F1       : {0:.3f} <- 2*((precision*recall)/(precision+recall))'.format(F1))

    # Registrar Resultados
    df_comparativa = df_comparativa.append({'Modelo': nombre_modelo,
                                            'Umbral': umbral,
                                            'Aciertos': aciertos,
                                            'Errores': errores,
                                            'Indeterm': indeterm,
                                            'Precision': precision,
                                            'Recall': recall,
                                            'Accuracy': accuracy,
                                            'F1': F1,
                                            'df_Aciertos': df_test[ ( df_test['intencion'] == df_test['Predic'] ) & ( df_test['Probabilidad'] >= umbral ) ],
                                            'df_Errores' : df_test[ ( df_test['intencion'] != df_test['Predic'] ) & ( df_test['Probabilidad'] >= umbral ) ],
                                            'df_Indeterm': df_test[ ( df_test['Probabilidad'] < umbral ) ]
                                            }, ignore_index=True)
    return

C:\Users\tomas\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Mejor accuracy: 0.6757
Mejor C: {'C': 1}


In [7]:
Evaluar_Modelo(grid_RLog, 'Regresión Logística Sin Regularización', X_test, y_test, umbral=0.8)

Cantidad de registros evaluados: 157
--------------------
Aciertos: 109
Errores : 35
Indeterm: 13
--------------------
Precision: 0.757 <- aciertos/(aciertos+errores)
Recall   : 0.893 <- aciertos/(aciertos+indeterm)
Accuracy : 0.777 <- (aciertos+indeterm)/(aciertos+errores+indeterm)
F1       : 0.820 <- 2*((precision*recall)/(precision+recall))


In [8]:
df_comparativa

,Modelo,Umbral,Aciertos,Errores,Indeterm,Precision,Recall,Accuracy,F1,df_Aciertos,df_Errores,df_Indeterm
0,Distancia coseno,0.8,100,36,21,0.735294,0.826446,0.770701,0.778210,oracion inte...,oracion...,oracion ...
1,Regresión Logística Sin Regularización,0.8,109,35,13,0.756944,0.893443,0.777070,0.819549,oracion intencion...,oracion...,oracion ...


In [40]:
#df_comparativa.iloc[0].df_Errores

In [41]:
#df_comparativa.iloc[1].df_Errores

In [9]:
RLog=LogisticRegression(penalty='none', max_iter=10000, tol=0.0001, multi_class='ovr',)

parametros_RLog = {'C':[1]}

grid_RLog1 = GridSearchCV(estimator = RLog,scoring = 'accuracy',param_grid = parametros_RLog, cv = 5, n_jobs = -1)

grid_RLog1.fit(X[X.columns[1:]].values, y)


AC_RLog_best=grid_RLog1.best_score_
print('Mejor accuracy: ' + str(round(AC_RLog_best,4)))

C_RLog_best=grid_RLog1.best_params_ 
print('Mejor C: ' + str(C_RLog_best))

C:\Users\tomas\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Mejor accuracy: 0.5976
Mejor C: {'C': 1}


In [10]:
Evaluar_Modelo(grid_RLog1, 'Regresión Logística Sin Regularización', X_test, y_test, umbral=0.8)

Cantidad de registros evaluados: 157
--------------------
Aciertos: 154
Errores : 0
Indeterm: 3
--------------------
Precision: 1.000 <- aciertos/(aciertos+errores)
Recall   : 0.981 <- aciertos/(aciertos+indeterm)
Accuracy : 1.000 <- (aciertos+indeterm)/(aciertos+errores+indeterm)
F1       : 0.990 <- 2*((precision*recall)/(precision+recall))


In [13]:
import pickle

# Grabar el modelo elegido
nombre_archivo='modelo_intents.sav'
pickle.dump(grid_RLog1, open(nombre_archivo, 'wb'))

# Grabar el vectorizador
nombre_archivo='vectorizador_intents.sav'
pickle.dump(vectorizador, open(nombre_archivo, 'wb'))

In [11]:
import pickle

# Grabar el modelo elegido

pickle.dump(grid_RLog1, open("modelo_intents.sav", "wb"))

# Grabar el vectorizador
pickle.dump(vectorizador, open("vectorizador_intents.pkl", "wb"))

pickle.dump(df_textos, open(r'df_matriz_intents.pkl', 'wb'))